In [7]:
import numpy as np 
import pandas as pd

In [2]:
export = ""

In [ ]:
eventParticipate = pd.read_csv("") #df with the events each thermostat participated in

events = pd.read_csv("") #start end times of events
events['START']=pd.to_datetime(events.START, utc=True).dt.tz_convert('America/Los_Angeles')
events['END']=pd.to_datetime(events.END, utc=True).dt.tz_convert('America/Los_Angeles')


eventsHours = pd.read_csv("")
eventsHours['DATE'] = pd.to_datetime(eventsHours.DATE, utc=True).dt.tz_convert('America/Los_Angeles')

eventsHours.head()

In [ ]:
finalDF = pd.DataFrame()
count = 0
for identifier in eventParticipate.Identifier.unique():

    dfKW = pd.read_csv() #dataframe with identifiers modeled electricity usage during events
    dfKW['DATE'] = pd.to_datetime(dfKW.DATE, utc=True).dt.tz_convert('America/Los_Angeles')
    dfKW = dfKW.set_index('DATE')
    
    eventsUse = eventParticipate[(eventParticipate.Identifier == identifier)&(eventParticipate.value == True)].event.unique()
    
    eventsDF = pd.DataFrame()
    
    for event in eventsUse:
        hoursDF = eventsHours[eventsHours.event == event]
        
        for startTime,hour,minute,event in zip(hoursDF.DATE,hoursDF.event_hour,hoursDF.minutes,hoursDF.event):
            if hour == 60:
                minuteAdd = 55
            else:
                minuteAdd = 50
                
            dfEventHour = dfKW[startTime:startTime+pd.Timedelta(minutes=minuteAdd)]

             
            dfNew = pd.DataFrame({'Identifier':[identifier],'DATE':[startTime],'event':[event],'hour_event':[hour],'minutes':[minute],
                                 'HourlyDryBulbTemperature':[dfEventHour.HourlyDryBulbTemperature.mean()],
                                  'expected_kWh':[dfEventHour.expected_kWh.sum()],'perfect_kWh':[dfEventHour.perfect_kWh.sum()],
                                 'actual_kWh':[dfEventHour.actual_kWh.sum()]})
            
            eventsDF = pd.concat([eventsDF,dfNew])
            
    finalDF = pd.concat([finalDF, eventsDF])        
    count += 1
    print(str(count))
            
finalDF['expected_kW'] = finalDF.expected_kWh * (60/finalDF.minutes)
finalDF['perfect_kW'] = finalDF.perfect_kWh * (60/finalDF.minutes)
finalDF['actual_kW'] = finalDF.actual_kWh * (60/finalDF.minutes)
    
finalDF['observed_impact_kW'] = finalDF.actual_kW - finalDF.expected_kW
finalDF['counterfactual_impact_kW'] = finalDF.perfect_kW - finalDF.expected_kW
finalDF['override_imapct_kW'] = finalDF.observed_impact_kW - finalDF.counterfactual_impact_kW
    
finalDF.to_csv(export + 'resultsByThermostatByHour.csv')

In [ ]:
results = pd.DataFrame()
count = 0
for identifier in finalDF.Identifier.unique():
    
    identifierDF = finalDF[finalDF.Identifier == identifier]
    
    for hourUse in identifierDF.hour_event.unique():
        
        hourDF = identifierDF[identifierDF.hour_event == hourUse]
        
        idDF = pd.DataFrame({'Identifier':[identifier],'hour_event':[hourUse],
                             'observed_impact_kW_std':[hourDF.observed_impact_kW.std()],
                            'counterfactual_impact_kW_std':[hourDF.counterfactual_impact_kW.std()],
                            'observed_impact_kW_mean':[hourDF.observed_impact_kW.mean()],
                            'counterfactual_impact_kW_mean':[hourDF.counterfactual_impact_kW.mean()]})
        results = pd.concat([results, idDF])
    count += 1
    print(str(count))
    
    results.to_csv(export + 'resultByHourStdevMean.csv')

In [ ]:
lolp = pd.read_csv("") #time series of LOLP by hour -- ELCC calculation output
identifiers = pd.read_csv() #resultByHourStdevMean.csv file fromm earlier
identifiers.head()

In [ ]:
melt = pd.melt(identifiers[['Identifier','hour_event','observed_impact_kW_mean','counterfactual_impact_kW_mean']],id_vars=['Identifier','hour_event'])
group = melt.groupby(['hour_event','variable'])['value'].sum().reset_index()
group.head()

In [ ]:
df = pd.pivot_table(group,values='value',index='hour_event',columns=['variable']).reset_index()
df.to_csv('')
df.head()

In [59]:
identifierProfileObserved = lolp.merge(df[['hour_event','observed_impact_kW_mean']],how='left',on='hour_event')
identifierProfileCounterfactual = lolp.merge(df[['hour_event','counterfactual_impact_kW_mean']],how='left',on='hour_event')

hours = [1,2,3,4]

for hour in hours:
    
    identifierProfileObservedX = identifierProfileObserved
    identifierProfileCounterfactualX = identifierProfileCounterfactual
    
    identifierProfileObservedX['delta_override'] = identifierProfileObservedX['observed_impact_kW_mean'].fillna(0)
    identifierProfileCounterfactualX['delta_noOverride'] = identifierProfileCounterfactualX['counterfactual_impact_kW_mean'].fillna(0)

    identifierProfileObservedX['delta_override'] = np.where(identifierProfileObservedX.hour_event != hour, 0, identifierProfileObservedX.delta_override)
    identifierProfileCounterfactualX['delta_noOverride'] = np.where(identifierProfileCounterfactualX.hour_event != hour, 0, identifierProfileCounterfactualX.delta_noOverride)
    
    finalObserved = identifierProfileObservedX.drop(columns=['LOLP','flag','hour_event','observed_impact_kW_mean'])
    finalCounterfactual = identifierProfileCounterfactualX.drop(columns=['LOLP','flag','hour_event','counterfactual_impact_kW_mean'])

    finalObserved['delta_override'] = finalObserved.delta_override * -1
    finalCounterfactual['delta_noOverride'] = finalCounterfactual.delta_noOverride * -1

    finalObserved.to_csv("")
    finalCounterfactual.to_csv('')